# Hybrid CNN + klassische Features – Storyline

## 1. Setup & Datensatz laden
- Kaggle-Datensatz importieren, erste Bilder anzeigen  
- Kurzer Blick auf Datenstruktur und Labels  

## 2. Baseline: reines CNN <-- Zu langsam auf CPU, Projekt vorerst hier eingefroren
- Kleines CNN in Keras trainieren („quick & dirty“)  
- Ergebnis/Accuracy dokumentieren → Ausgangsbasis  

## 3. Klassische Features extrahieren (HOG, DoG, Textur-Histogramme)
- Mit `scikit-image` / `opencv` Feature-Maps generieren  
- Visualisierung: Beispielbilder mit klassischen Merkmalen

## 4. Hybrid-Modell aufbauen
- CNN-Features + klassische Features kombinieren (z. B. Concatenate in Dense Layer)  
- Training starten, Ergebnisse vergleichen  

## 5. Vergleich & Analyse
- Accuracy/Validation Loss: „nur CNN“ vs. „Hybrid CNN + klassische Features“  
- Kleine Tabelle oder Diagramm mit Ergebnissen  

## 6. Fazit
- Hybridansatz liefert oft *good enough* Ergebnisse  
- Vorteil: weniger Filter, weniger Datenbedarf, robustere Generalisierung

In [1]:
import kagglehub
import os
import numpy as np
import tensorflow as tf
from ImageLoading import load_and_tile, get_number_of_tiles
from Models import build_damage_cnn

path = kagglehub.dataset_download("angelikafarahmanoppo/road-surface-dataset-datasetjalan")

2025-09-03 17:26:15.934089: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 17:26:15.934350: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 17:26:15.965831: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-03 17:26:16.626995: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

In [2]:
SEED: int = 42
TILE_SIZE: int = 256



In [3]:
path_class_1: str = path + "/road_surface_dataset/Jalan Kategori Baik/"
path_class_2: str = path + "/road_surface_dataset/Jalan Kurang Baik"
path_class_3: str = path + "/road_surface_dataset/Jalan Rusak"
directory_paths = [path_class_1, path_class_2, path_class_3]
labels = [0.0, 0.5, 1.0]
total_tiles = get_number_of_tiles(directory_paths, TILE_SIZE, TILE_SIZE)
print(f"Total number of tiles: {total_tiles=}")
train_size = int(total_tiles * 0.7)   # 70% for training -> set to 30% for test
val_size = int(total_tiles * 0.15)  # 15% for validation -> rest for testing

Total number of tiles: total_tiles=50526


In [4]:
dataset = tf.data.Dataset.from_generator(
    lambda: load_and_tile(directory_paths, labels, TILE_SIZE, TILE_SIZE),
    output_signature = (
        tf.TensorSpec(shape=(TILE_SIZE, TILE_SIZE, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )
)

dataset = dataset.shuffle(1000, seed=SEED).batch(32).prefetch(tf.data.AUTOTUNE)

2025-09-03 17:26:44.745859: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
train_dataset = dataset.take(train_size).cache().prefetch(tf.data.AUTOTUNE).repeat()
val_dataset = dataset.skip(train_size).take(val_size).cache().prefetch(tf.data.AUTOTUNE)
test_dataset = dataset.skip(train_size + val_size).cache().prefetch(tf.data.AUTOTUNE)

In [6]:
model = build_damage_cnn((TILE_SIZE,TILE_SIZE,1))
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
check_point = tf.keras.callbacks.ModelCheckpoint(
    file_path
)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=50, steps_per_epoch=train_size, validation_steps=val_size, callbacks=[early_stop])
model.save("damage_cnn.keras")
# model = tf.keras.models.load_model("damage_cnn.keras")

Epoch 1/50
35368/35368 ━━━━━━━━━━━━━━━━━━━━ 9905s 280ms/step - loss: 0.0239 - mae: 0.0609
Epoch 2/50


2025-09-03 20:11:50.228459: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/dierck/workspace/Demos/Notebooks/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/home/dierck/workspace/Demos/Notebooks/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)


35368/35368 ━━━━━━━━━━━━━━━━━━━━ 9795s 277ms/step - loss: 0.0313 - mae: 0.0839
Epoch 3/50


2025-09-03 22:55:04.752421: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


35368/35368 ━━━━━━━━━━━━━━━━━━━━ 9777s 276ms/step - loss: 0.0303 - mae: 0.0795
Epoch 4/50
35368/35368 ━━━━━━━━━━━━━━━━━━━━ 9787s 277ms/step - loss: 0.0288 - mae: 0.0759
Epoch 5/50


2025-09-04 04:21:08.384963: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13741/35368 ━━━━━━━━━━━━━━━━━━━━ 1:39:48 277ms/step - loss: 0.0294 - mae: 0.0769

KeyboardInterrupt: 

In [ ]:
results = model.predict(test_dataset)